In [7]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import MultiLabelBinarizer
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import keras
from keras.applications import EfficientNetB4, ResNet50, DenseNet121
from keras import layers, models
from keras.metrics import AUC, Precision, Recall
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import mixed_precision

# =========================
# Mixed precision (fp16)
# =========================
mixed_precision.set_global_policy("mixed_float16")

# ------------------------------------------------------------
# 1) Caminhos e hiperparâmetros
# ------------------------------------------------------------
DATA_DIR = Path("../data")  # ajuste se necessário
TRAIN_CSV = DATA_DIR / "train.csv"
TRAIN_DIR = DATA_DIR / "train_images"

IMG_SIZE = (224, 224)
BATCH_SIZE = 64
SEED = 42
EPOCHS = 10
AUTOTUNE = tf.data.AUTOTUNE
rng = np.random.default_rng(SEED)

# ------------------------------------------------------------
# 2) Carregar labels e montar vetor multi-hot
# ------------------------------------------------------------
df = pd.read_csv(TRAIN_CSV)
df["labels"] = df["labels"].astype(str).str.strip().str.split()
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["labels"]).astype("float32")
X = df["image"].values
class_names = list(mlb.classes_)
num_classes = len(class_names)
print("Classes:", class_names)

# ------------------------------------------------------------
# 3) tf.data helpers (leitura, resize, normalização, augment)
# ------------------------------------------------------------
# Augment na GPU
data_augment = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.1),
    ],
    name="augment",
)


def _load_image(path):
    img = tf.io.read_file(path)
    # use decode_png se suas imagens forem .png
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE, antialias=True)
    img = tf.cast(img, tf.float32) / 255.0
    return img


def make_ds(paths, labels, training: bool):
    """Cria um tf.data.Dataset de (image, multi_hot) com pipeline rápido."""
    paths = tf.convert_to_tensor(paths)
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        # embaralha bem (não gigante pra não explodir RAM)
        buffer = min(10000, len(paths))
        ds = ds.shuffle(buffer, seed=SEED, reshuffle_each_iteration=True)

    # paraleliza leitura/decodificação
    ds = ds.map(lambda p, y: (_load_image(p), y), num_parallel_calls=AUTOTUNE)

    ds = ds.cache(
        f".cache/cached_{'train' if training else 'val'}"
    )  # habilite se couber na RAM/SSD (ou use .cache('arquivo'))

    if training:
        ds = ds.map(
            lambda x, y: (data_augment(x, training=True), y),
            num_parallel_calls=AUTOTUNE,
        )

    ds = ds.batch(BATCH_SIZE, drop_remainder=False)
    ds = ds.prefetch(AUTOTUNE)

    # para máximo throughput
    options = tf.data.Options()
    options.experimental_deterministic = False
    ds = ds.with_options(options)
    return ds


# ------------------------------------------------------------
# helper: cria modelo (novo a cada fold)
# ------------------------------------------------------------
def build_model(num_classes: int):
    # base_model = ResNet50(
    #     weights="imagenet", include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    # )
    base_model = EfficientNetB4(
        weights="imagenet", include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )
    # base_model = DenseNet121(
    #     weights="imagenet", include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    # )
    base_model.trainable = False  # fase 1: só a cabeça

    inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    pos = y.sum(axis=0)  # positivos por classe
    neg = y.shape[0] - pos
    prior = np.clip(pos / (pos + neg), 1e-6, 1 - 1e-6)
    bias_init = np.log(prior / (1 - prior))
    outputs = layers.Dense(
        num_classes,
        activation="sigmoid",
        bias_initializer=keras.initializers.Constant(bias_init),
        dtype="float32",
    )(x)

    model = models.Model(inputs, outputs)
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=[
            "accuracy",
            Precision(name="precision", thresholds=0.25),
            Recall(name="recall", thresholds=0.25),
            AUC(name="auc", multi_label=True),
        ],
    )
    return model


# ------------------------------------------------------------
# 4) K-Fold Cross-Validation (4 folds) com tf.data
# ------------------------------------------------------------
mskf = MultilabelStratifiedKFold(n_splits=4, shuffle=True, random_state=SEED)
fold_metrics = []

os.makedirs(".cache", exist_ok=True)

for fold, (train_idx, val_idx) in enumerate(mskf.split(X, y), start=1):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx].astype("float32"), y[val_idx].astype("float32")

    X_train_paths = [str(TRAIN_DIR / fname) for fname in X_train]
    X_val_paths = [str(TRAIN_DIR / fname) for fname in X_val]

    train_ds = make_ds(X_train_paths, y_train, training=True)
    val_ds = make_ds(X_val_paths, y_val, training=False)

    # modelo novo por fold
    model = build_model(num_classes)
    if fold == 1:
        model.summary()

    callbacks = [
        EarlyStopping(
            monitor="val_loss", patience=4, min_delta=0.003, restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor="val_loss", factor=0.2, patience=2, min_delta=0.005, min_lr=1e-6
        ),
        ModelCheckpoint(
            f"best_fold{fold}.keras", monitor="val_loss", save_best_only=True
        ),
    ]

    print(f"\n===== FOLD {fold}/4 =====")

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        verbose=1,
        callbacks=callbacks,
    )

    # Avaliação neste fold
    fold_result = model.evaluate(val_ds, return_dict=True, verbose=1)
    fold_result["fold"] = fold
    fold_metrics.append(fold_result)

    print(
        f"Fold {fold} -> "
        f"val_loss={fold_result['loss']:.4f} | "
        f"val_acc={fold_result['accuracy']:.4f} | "
        f"val_prec={fold_result['precision']:.4f} | "
        f"val_rec={fold_result['recall']:.4f} | "
        f"val_auc={fold_result['auc']:.4f}"
    )

    # Limpa cache para o próximo fold
    keras.backend.clear_session()

    for arquivo in os.listdir(".cache"):
        caminho_arquivo = os.path.join(".cache", arquivo)
        if os.path.isfile(caminho_arquivo):
            os.remove(caminho_arquivo)


# ------------------------------------------------------------
# 5) Resumo dos 4 folds (média ± desvio)
# ------------------------------------------------------------
fold_df = pd.DataFrame(fold_metrics).set_index("fold")
print("\nResultados por fold:")
print(fold_df.round(4))

summary = fold_df.agg(["mean", "std"]).round(4)
print("\nMédia e desvio (4 folds):")
print(summary)

nome_modelo = model.layers[1].name
fold_df.to_csv(f"../results/results_{nome_modelo}.csv", float_format="%.4f")

# ------------------------------------------------------------
# (Opcional) Fine-tuning por fold
# ------------------------------------------------------------
# Depois da fase inicial, você pode descongelar parte da ResNet e rodar mais épocas:
# base_model = model.layers[1]  # se usar Model(inputs, outputs), ajuste índice conforme seu grafo
# base_model.trainable = True
# for layer in base_model.layers[:-30]:
#     layer.trainable = False
# model.compile(
#     optimizer=keras.optimizers.Adam(1e-5),
#     loss="binary_crossentropy",
#     metrics=["accuracy", Precision(name="precision"), Recall(name="recall"),
#              AUC(name="auc", multi_label=True)]
# )
# model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FT, callbacks=callbacks)

Classes: ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb4 (Functional)     │ (None, 7, 7, 1792)     │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1792)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       459,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,134,373 (69.18 MB)

 Trainable params: 460,550 (1.76 MB)

...

219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step - accuracy: 0.2491 - auc: 0.5001 - loss: 0.4515 - precision: 0.2765 - recall: 0.4007

2025-10-05 12:17:18.591766: W tensorflow/core/kernels/data/cache_dataset_ops.cc:333] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-10-05 12:17:24.995154: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-10-05 12:17:25.195524: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-10-05 12:17:25.582053: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel t

219/219 ━━━━━━━━━━━━━━━━━━━━ 184s 714ms/step - accuracy: 0.2491 - auc: 0.5001 - loss: 0.4515 - precision: 0.2765 - recall: 0.4009 - val_accuracy: 0.2568 - val_auc: 0.5000 - val_loss: 0.4472 - val_precision: 0.2788 - val_recall: 0.5122 - learning_rate: 0.0010
Epoch 2/10


2025-10-05 12:17:40.390437: W tensorflow/core/kernels/data/cache_dataset_ops.cc:333] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


219/219 ━━━━━━━━━━━━━━━━━━━━ 37s 168ms/step - accuracy: 0.2589 - auc: 0.4989 - loss: 0.4476 - precision: 0.2946 - recall: 0.3176 - val_accuracy: 0.2568 - val_auc: 0.5000 - val_loss: 0.4472 - val_precision: 0.3082 - val_recall: 0.2831 - learning_rate: 0.0010
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 37s 167ms/step - accuracy: 0.2589 - auc: 0.4942 - loss: 0.4476 - precision: 0.3051 - recall: 0.2827 - val_accuracy: 0.2568 - val_auc: 0.5000 - val_loss: 0.4472 - val_precision: 0.3082 - val_recall: 0.2831 - learning_rate: 0.0010
Epoch 4/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 37s 168ms/step - accuracy: 0.2589 - auc: 0.4991 - loss: 0.4476 - precision: 0.3053 - recall: 0.2821 - val_accuracy: 0.2568 - val_auc: 0.5000 - val_loss: 0.4472 - val_precision: 0.3082 - val_recall: 0.2831 - learning_rate: 2.0000e-04
Epoch 5/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 36s 162ms/step - accuracy: 0.2589 - auc: 0.4990 - loss: 0.4476 - precision: 0.3053 - recall: 0.2821 - val_accuracy: 0.2568 - val_auc: 0.5000 - val_loss: 0.4472 